In [1]:
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
from src.wilson import * 

## Parameters

In [2]:
L = 4

In [3]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges) # note hilbert space is on edges
# there is no natural ordering to the hilbert space so we will have to impose one.
# this is specified by wilson.ordered_edges.
# as long as we respect this ordering across all implementations, things should work

g.n_edges = 32, LxLx2 = 32


$$
H = -K \sum_{\square} \prod_{ij \in \square} \sigma_{ij}^z - g \prod_{ij} \sigma_{ij}^x
$$

In [4]:
g.edges()

[(3, 7),
 (12, 13),
 (8, 9),
 (8, 12),
 (2, 14),
 (13, 14),
 (4, 5),
 (5, 6),
 (4, 8),
 (12, 15),
 (5, 9),
 (14, 15),
 (3, 15),
 (8, 11),
 (0, 1),
 (9, 10),
 (1, 2),
 (0, 4),
 (9, 13),
 (10, 11),
 (1, 5),
 (10, 14),
 (6, 7),
 (6, 10),
 (4, 7),
 (0, 3),
 (0, 12),
 (2, 3),
 (1, 13),
 (2, 6),
 (11, 15),
 (7, 11)]

In [13]:
ord_edges = ordered_edges(g)
i = 15
ord_edges[i + L**2], ord_edges[i]
# missing_edges = []
# for e in g.edges():
# 	if not (e in ord_edges or (e[1], e[0]) in ord_edges):
# 		missing_edges.append(e)
# missing_edges
# print(ord_edges)
# (i + 1) % (L-1)

left = ord_edges[i] 
right = ord_edges[i + 1]
upper = ord_edges[i + L**2]
lower = ord_edges[(i + L**2 + L) % (L**2)] # <--- this goes out of range
left, right, upper, lower

((15, 3), (0, 1), (15, 12), (3, 7))

Equivariant layer: Two channels (I think for 2 valued dof), stride 1 CNN with leaky relu then wilson path pointwise multiplication, and skip connection.
Invariant layer: two channels, stride 1,. 
Network with equivariant layer tends to do better.